# Convert recorded Json file data to Sqlite
Still a bit trail and error, but the Json format is fixed enough to start working on full file conversion and upload into the Sqlite database.

In [1]:
from datetime import datetime
import json
import sqlite3
import pandas as pd

In [2]:
#sqlSchemaInitialisation = open('RawData/developmentSchema03.sql', encoding='utf8').read()
##print(sqlSchemaInitialisation)
#cnx = sqlite3.connect(":memory:")
#cur = cnx.cursor()
#print(cnx)
#print(cur)
#cur.executescript(sqlSchemaInitialisation)

In [3]:
def getKnownLeaders(cnx):
    cur = cnx.cursor()
    sqlSelect = 'SELECT leaderId, name, civ FROM leader'
    return pd.read_sql_query(sqlSelect, cnx)

def getLeaderId(name, civ, knownLeaders):
    try:
        # The iloc is neccessary as we can't assume there is only one return value!?
        # also have to make sure it is int as numpy.int64 doesn't go into sqlite so well?
        return int(knownLeaders[(knownLeaders.name == name) & (knownLeaders.civ == civ)].iloc[0].leaderId)
    except IndexError:
        return None

In [4]:
jsonData = open('RawData/20181128_01.json', encoding='utf8').read()

In [5]:
rawData = json.loads(jsonData)

In [ ]:
cnx = sqlite3.connect('Database/Civ6CitySettledData.db')
cur = cnx.cursor()
print(cnx)
print(cur)

## Populate the game table

In [6]:
# Python creates a list by default, we know we only have 1 so select first element only!
gameData = rawData['gameConfig'][0]
print(gameData)
print(type(gameData))

{'difficulty': 'Prince', 'ruleSet': 'RULESET_EXPANSION_1', 'mapType': 'ToDo', 'startEra': 'Ancient Era', 'players': [{'playerCiv': 'France', 'playerName': 'Catherine de Medici'}, {'playerCiv': 'Scythia', 'playerName': 'Tomyris'}, {'playerCiv': 'Spain', 'playerName': 'Philip II'}, {'playerCiv': 'Arabia', 'playerName': 'Saladin'}, {'playerCiv': 'Sumeria', 'playerName': 'Gilgamesh'}, {'playerCiv': 'Germany', 'playerName': 'Frederick Barbarossa'}, {'playerCiv': 'Babylon', 'playerName': 'Babylon'}, {'playerCiv': 'Mohenjo-Daro', 'playerName': 'Mohenjo-Daro'}, {'playerCiv': 'Kumasi', 'playerName': 'Kumasi'}, {'playerCiv': 'Toronto', 'playerName': 'Toronto'}, {'playerCiv': 'Granada', 'playerName': 'Granada'}, {'playerCiv': 'Muscat', 'playerName': 'Muscat'}, {'playerCiv': 'Bandar Brunei', 'playerName': 'Bandar Brunei'}, {'playerCiv': 'Kandy', 'playerName': 'Kandy'}, {'playerCiv': 'Brussels', 'playerName': 'Brussels'}], 'mapSize': 'Small', 'leader': 'Catherine de Medici', 'gameSpeed': 'Standard'

In [7]:
dateString = rawData['date']
date = datetime.strptime(dateString, '%Y%m%d').date()
game = {
    'gameId': None,
    'date': date,
    'difficulty': gameData['difficulty'],
    'mapType': 'Continents',
    'mapSize': gameData['mapSize'],
    'gameSpeed': gameData['gameSpeed'],
    'startEra': gameData['startEra'],
    'ruleSet': gameData['ruleSet'],
}
print(game)

{'gameId': None, 'date': datetime.date(2018, 11, 28), 'difficulty': 'Prince', 'mapType': 'Continents', 'mapSize': 'Small', 'gameSpeed': 'Standard', 'startEra': 'Ancient Era', 'ruleSet': 'RULESET_EXPANSION_1'}


In [8]:
cur.execute('INSERT INTO game VALUES(:gameId, :date, :difficulty, :mapType, :mapSize, :gameSpeed, :startEra, :ruleSet)',
    game)
gameId = cur.lastrowid
print("gameID: ", gameId)

gameID:  6


In [9]:
cnx.commit()

## Add "new" leaders to the leader table.
The leader table is a list of all the leaders we've captured data for, regardless of when they were first seen. We record the relationship between a specific game/instance with the leaders in the gameLeader table.

In [10]:
knownLeaders = getKnownLeaders(cnx)
print(knownLeaders)

    leaderId                  name            civ
0          7             Amanitore          Nubia
1         35               Antioch        Antioch
2         14                Armagh         Armagh
3         28              Auckland       Auckland
4          4               Babylon        Babylon
5          3         Bandar Brunei  Bandar Brunei
6         13              Brussels       Brussels
7         47          Buenos Aires   Buenos Aires
8         48              Carthage       Carthage
9         11   Catherine de Medici         France
10        22             Cleopatra          Egypt
11        26                 Cyrus         Persia
12        51  Frederick Barbarossa        Germany
13        12                Geneva         Geneva
14        21          Genghis Khan       Mongolia
15         1             Gilgamesh        Sumeria
16        20               Gitarja      Indonesia
17        45               Granada        Granada
18        24       Harald Hardrada         Norway


In [12]:
result = []
for leader in gameData['players']:
    name = leader['playerName']
    civ = leader['playerCiv']
    leaderId = getLeaderId(name, civ, knownLeaders)
    if leaderId is None:
        newLeader = {
            'leaderId': None,
            'name': name,
            'civ': civ,
            'isMajor': False if (name == civ) else True
        }
        result.append(newLeader)

for row in result:
    print(row)

In [13]:
for row in result:
#    print(row)
    cur.execute('INSERT INTO leader VALUES(:leaderId, :name, :civ, :isMajor)', row)

In [14]:
cnx.commit()

# Populate gameLeader table

In [15]:
knownLeaders = getKnownLeaders(cnx)
print(knownLeaders)
humanLeaderName = gameData['leader']
print(humanLeaderName)
print(gameId)

    leaderId                  name            civ
0          7             Amanitore          Nubia
1         35               Antioch        Antioch
2         14                Armagh         Armagh
3         28              Auckland       Auckland
4          4               Babylon        Babylon
5          3         Bandar Brunei  Bandar Brunei
6         13              Brussels       Brussels
7         47          Buenos Aires   Buenos Aires
8         48              Carthage       Carthage
9         11   Catherine de Medici         France
10        22             Cleopatra          Egypt
11        26                 Cyrus         Persia
12        51  Frederick Barbarossa        Germany
13        12                Geneva         Geneva
14        21          Genghis Khan       Mongolia
15         1             Gilgamesh        Sumeria
16        20               Gitarja      Indonesia
17        45               Granada        Granada
18        24       Harald Hardrada         Norway


In [16]:
result = []
for leader in gameData['players']:
    name = leader['playerName']
    civ = leader['playerCiv']
    leaderId = getLeaderId(name, civ, knownLeaders)
    if name == humanLeaderName:
        newGameLeader = {
            'gameId': gameId,
            'leaderId': leaderId,
            'isHuman': True,
        }
        result.append(newGameLeader)
    else:
        newGameLeader = {
            'gameId': gameId,
            'leaderId': leaderId,
            'isHuman': False,
        }
        result.append(newGameLeader)
    
for row in result:
    print(row)

{'gameId': 6, 'leaderId': 11, 'isHuman': True}
{'gameId': 6, 'leaderId': 27, 'isHuman': False}
{'gameId': 6, 'leaderId': 49, 'isHuman': False}
{'gameId': 6, 'leaderId': 50, 'isHuman': False}
{'gameId': 6, 'leaderId': 1, 'isHuman': False}
{'gameId': 6, 'leaderId': 51, 'isHuman': False}
{'gameId': 6, 'leaderId': 4, 'isHuman': False}
{'gameId': 6, 'leaderId': 19, 'isHuman': False}
{'gameId': 6, 'leaderId': 52, 'isHuman': False}
{'gameId': 6, 'leaderId': 53, 'isHuman': False}
{'gameId': 6, 'leaderId': 45, 'isHuman': False}
{'gameId': 6, 'leaderId': 33, 'isHuman': False}
{'gameId': 6, 'leaderId': 3, 'isHuman': False}
{'gameId': 6, 'leaderId': 34, 'isHuman': False}
{'gameId': 6, 'leaderId': 13, 'isHuman': False}


In [17]:
for row in result:
#    print(row)
    cur.execute('INSERT INTO gameLeader VALUES(:gameId, :leaderId, :isHuman)', row)

In [18]:
cnx.commit()

# Populate citySettled and gameCity Tables

In [21]:
citySettledLog = rawData['cityOnSettledLog']
#print(citySettledLog)
for city in citySettledLog:
    ownerName = city['ownerName']
    name = city['cityName']
    leaderId = int(knownLeaders[(knownLeaders.name == ownerName)].iloc[0].leaderId)

    sqlInsert = "INSERT INTO citySettledLog VALUES(:cityId, :name, :settledById, :onTurn)"
    parameters = {'cityId': None, 'name': name, 'settledById':leaderId, 'onTurn':city['turn']}
    cur.execute(sqlInsert, parameters)
    cityId = cur.lastrowid
    print(cityId, ownerName, name)
    
    sqlInsert = "INSERT INTO gameCity VALUES(:gameId, :cityId)"
    parameters = {'gameId':gameId, 'cityId':cityId}
    cur.execute(sqlInsert, parameters)

84 Philip II Madrid
85 Gilgamesh Uruk
86 Babylon Babylon
87 Mohenjo-Daro Mohenjo-Daro
88 Kumasi Kumasi
89 Toronto Toronto
90 Granada Granada
91 Muscat Muscat
92 Bandar Brunei Bandar Brunei
93 Kandy Kandy
94 Brussels Brussels
95 Catherine de Medici Paris
96 Tomyris Pokrovka
97 Frederick Barbarossa Aachen


In [20]:
#cnx.rollback()

In [22]:
cnx.commit()

## Populate cityPlotsSettled

In [23]:
sqlSelect = 'SELECT * FROM gameCityLeaderSettledView WHERE gameId = :gameId'
gameCLS = pd.read_sql_query(sqlSelect, cnx, params={'gameId':gameId})
print(gameCLS)

    gameId  cityId       cityName            leaderName  leaderId  \
0        6      84         Madrid             Philip II        49   
1        6      85           Uruk             Gilgamesh         1   
2        6      86        Babylon               Babylon         4   
3        6      87   Mohenjo-Daro          Mohenjo-Daro        19   
4        6      88         Kumasi                Kumasi        52   
5        6      89        Toronto               Toronto        53   
6        6      90        Granada               Granada        45   
7        6      91         Muscat                Muscat        33   
8        6      92  Bandar Brunei         Bandar Brunei         3   
9        6      93          Kandy                 Kandy        34   
10       6      94       Brussels              Brussels        13   
11       6      95          Paris   Catherine de Medici        11   
12       6      96       Pokrovka               Tomyris        27   
13       6      97         Aachen 

In [ ]:
# Testing, can safely ignore this!
#ownerCityId = int(gameCLS[gameCLS.cityName == 'Amsterdam'].iloc[0].cityId)
#print(ownerCityId)
#print(type(ownerCityId))

In [24]:
# building up the cityPlotsSettled data for entry...
result = []
for city in rawData['cityOnSettledLog']:
    for plot in city['plots']:
        try:
            ownerCityId = int(gameCLS[gameCLS.cityName == plot['ownerCity']].iloc[0].cityId)
        except IndexError:
            ownerCityId = None
        plotInfo = {
            'plotId': None,
            'ownerCityId': ownerCityId,
            'ring': plot['r'],
            'terrain': plot['terrain'],
            'feature': plot['feature'],
            'resource': plot['resource'],
            'resourceCount': plot['resourceCount'],
            'resourceType': plot['resourceType'],
            'workers': plot['workers'],
            'district': plot['district'],
            'hasRiver': plot['hasRiver'],
            'isWater': plot['isWater'],
            'isLake': plot['isLake'],
            'isCity': plot['isCity'],
        }
        result.append(plotInfo)

print(result[0])
print(result[1])
print(result[2])

{'plotId': None, 'ownerCityId': 84, 'ring': 0, 'terrain': 'Plains (Hills)', 'feature': 'None', 'resource': 'None', 'resourceCount': 0, 'resourceType': 'None', 'workers': 1, 'district': 'City Center', 'hasRiver': True, 'isWater': False, 'isLake': False, 'isCity': True}
{'plotId': None, 'ownerCityId': 84, 'ring': 1, 'terrain': 'Grassland (Hills)', 'feature': 'None', 'resource': 'Stone', 'resourceCount': 1, 'resourceType': 'Bonus', 'workers': 0, 'district': 'None', 'hasRiver': True, 'isWater': False, 'isLake': False, 'isCity': False}
{'plotId': None, 'ownerCityId': 84, 'ring': 1, 'terrain': 'Plains (Hills)', 'feature': 'None', 'resource': 'Sheep', 'resourceCount': 1, 'resourceType': 'Bonus', 'workers': 0, 'district': 'None', 'hasRiver': True, 'isWater': False, 'isLake': False, 'isCity': False}


In [25]:
for row in result:
    cur.execute('INSERT INTO cityPlotsSettled VALUES(:plotId, :ownerCityId, :ring, :terrain, :feature, :resource, :resourceCount, :resourceType, :workers, :district, :hasRiver, :isWater, :isLake, :isCity )',
                row)

In [ ]:
#cnx.rollback()

In [26]:
cnx.commit()

## Populate cityPerTurnLog

In [27]:
def getCityId(cityName):
    try:
        # The iloc is neccessary as we can't assume there is only one return value!?
        # also have to make sure it is int as numpy.int64 doesn't go into sqlite so well?
        return int(gameCLS[gameCLS.cityName == cityName].iloc[0].cityId)
    except IndexError:
        return None
    
def getLeaderId(ownerName, ownerCiv):
    try:
        # The iloc is necessary as we can't assume there is only one return value!?
        # also have to make sure it is int as numpy.int64 doesn't go into sqlite so well?
        return int(gameCLS[gameCLS.leaderName  == ownerName].iloc[0].leaderId)
    except IndexError:
        return None

In [28]:
# building up the cityPerTurnLog data for entry...
result = []
for logEntry in rawData['cityPerTurnLog']:
    turnInfo = {
        'cityId': getCityId(logEntry['cityName']),
        'turn': int(logEntry['turn']),
        'ownerId': getLeaderId(logEntry['ownerName'], 'Blah'),
        'foodPerTurn': logEntry['foodPerTurn'],
        'foodToolTip': logEntry['foodToolTip'],
        'productionPerTurn': logEntry['productionPerTurn'],
        'productionToolTip': logEntry['productionToolTip'],
        'goldPerTurn': logEntry['goldPerTurn'],
        'goldToolTip': logEntry['goldToolTip'],
        'sciencePerTurn': logEntry['sciencePerTurn'],
        'scienceToolTip': logEntry['scienceToolTip'],
        'culturePerTurn': logEntry['culturePerTurn'],
        'cultureToolTip': logEntry['cultureToolTip'],
        'faithPerTurn': logEntry['faithPerTurn'],
        'faithToolTip': logEntry['faithToolTip'],
        'population': logEntry['population'],
        'housing': logEntry['housing'],
        'amenities': logEntry['amenities'],
        'amenitiesNeeded': logEntry['amenitiesNeeded'],
        'happiness': logEntry['happiness'],
    }
    result.append(turnInfo)

print(result[0])
print(result[1])
print(result[-1])

{'cityId': 84, 'turn': 1, 'ownerId': 49, 'foodPerTurn': '4.00', 'foodToolTip': '+4 from Worked Tiles', 'productionPerTurn': '6.25', 'productionToolTip': '+3 from Worked Tiles[NEWLINE]+2 from Buildings[NEWLINE]   +2 from Palace[NEWLINE]+20pct (+1) from Modifiers[NEWLINE]+5pct (+0.2) from Amenities', 'goldPerTurn': '7.50', 'goldToolTip': '+1 from Worked Tiles[NEWLINE]+5 from Buildings[NEWLINE]   +5 from Palace[NEWLINE]+20pct (+1.1) from Modifiers[NEWLINE]+5pct (+0.2) from Amenities', 'sciencePerTurn': '3.95', 'scienceToolTip': '+1 from Worked Tiles[NEWLINE]+2 from Buildings[NEWLINE]   +2 from Palace[NEWLINE]+0.5 from Population[NEWLINE]+8pct (+0.2) from Modifiers[NEWLINE]+5pct (+0.1) from Amenities', 'culturePerTurn': '1.46', 'cultureToolTip': '+1 from Buildings[NEWLINE]   +1 from Palace[NEWLINE]+0.2 from Population[NEWLINE]+8pct (+0.1) from Modifiers[NEWLINE]+5pct (+0) from Amenities', 'faithPerTurn': '0.00', 'faithToolTip': '+8pct (+0) from Modifiers[NEWLINE]+5pct (+0) from Amenities',

In [29]:
for row in result:
    cur.execute('INSERT INTO cityPerTurnLog VALUES(:cityId, :turn, :ownerId, :foodPerTurn, :foodToolTip, :productionPerTurn, :productionToolTip, :goldPerTurn, :goldToolTip, :sciencePerTurn, :scienceToolTip, :culturePerTurn, :cultureToolTip, :faithPerTurn, :faithToolTip, :population, :housing, :amenities, :amenitiesNeeded, :happiness)',
                row)

In [ ]:
cnx.rollback()

In [30]:
cnx.commit()

In [31]:
cnx.close()